In [2]:
import Pkg
Pkg.activate(".")

dependencies = [
    "Turing",
    "MCMCChains",
    "Distributions",
    "Bijectors",
    "Random",
    "ParetoSmooth",
    "Plots",            
    "StatsPlots",       
    "Statistics",       
    "LinearAlgebra",   
    "DataFrames",
    "NamedArrays", 
    "StatsFuns",
    "LogExpFunctions",        
    "JLD2",            
    "CSV"
]

Pkg.add(dependencies)

  Activating project at `c:\Users\michi\WorkSpace\gitwork\mich2000jp\hmm`
    Updating registry at `C:\Users\michi\.julia\registries\General.toml`
┌ Error: Some registries failed to update:
│     — C:\Users\michi\.julia\registries\General.toml — failed to download from https://pkg.julialang.org/registry/23338594-aafe-5451-b93e-139f81909106/1a0a48ee97065c291904c513a92d3be6802d36e1. Exception: RequestError: Recv failure: Connection was reset while requesting https://pkg.julialang.org/registry/23338594-aafe-5451-b93e-139f81909106/1a0a48ee97065c291904c513a92d3be6802d36e1
└ @ Pkg.Registry C:\Users\michi\.julia\juliaup\julia-1.12.1+0.x64.w64.mingw32\share\julia\stdlib\v1.12\Pkg\src\Registry\Registry.jl:551
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\michi\WorkSpace\gitwork\mich2000jp\hmm\Project.toml`
    Manifest No packages added to or removed from `C:\Users\michi\WorkSpace\gitwork\mich2000jp\hmm\Manifest.toml`


In [3]:
using Turing, Distributions, LinearAlgebra, Random, Bijectors, DataFrames, JLD2, MCMCChains, StatsPlots, ParetoSmooth, Logging
using StatsFuns: logistic, log1pexp

# Check number of threads
println("Running on ", Threads.nthreads(), " threads.")
# Set random seed
const SEED = 123;

Running on 12 threads.


In [ ]:
include("DataPreparation.jl")
FIX_PATH = "data/fix.csv"
RAND_PATH = "data/rand.csv"
DATA_PATH = "data/data.jld2"
# prepare_data(FIX_PATH, RAND_PATH, DATA_PATH)
data_fp = load_data(DATA_PATH, condition=:FP)
data_sp = load_data(DATA_PATH, condition=:SP);

In [60]:
n_iter     = 2000
n_burnin   = 2000
n_chains   = 4

trt        = "fp"
model_name = "iohmm"
K_states   = 2
prefix     = "_fin"

sampler    = NUTS(0.8; adtype=AutoForwardDiff())
println("=== Analysis Settings ===")
println("MCMC: $n_iter samples, $n_burnin burn-in, $n_chains chains")
println("Model: $model_name, K=$K_states, Condition=$trt, Prefix=$prefix")

=== Analysis Settings ===
MCMC: 2000 samples, 2000 burn-in, 4 chains
Model: iohmm, K=2, Condition=fp, Prefix=_fin


In [ ]:
title = "$(trt)_$(model_name)_K$(K_states)$prefix"
CHAIN_PATH    = "chain/$title.jld2"
SUMMARY_PATH = "output/summary_$title.csv"
LOO_PATH     = "output/loo_$title.csv"
PLOT_PATH     = "output/plot_$title.png"
PLOT_GQ_PATH  = "output/plot_$(title)_gq.png"
POST_PATH =[SUMMARY_PATH, LOO_PATH, PLOT_PATH, PLOT_GQ_PATH]

data = trt == "fp" ? data_fp : data_sp

include("models.jl")
include("mcmc_utils.jl")
model, model_gq = model_selected(model_name, data, K_states)


@load CHAIN_PATH chain

1-element Vector{Symbol}:
 :chain

In [ ]:
include("mcmc_utils.jl")
chain_relabeled = relabel_chain(chain, K_states)
loo = RunPSISLOO(model, chain_relabeled)
pareto_k = loo.pointwise(statistic = :pareto_k) |> collect
histogram(
    pareto_k;
    bins = :auto,
    xlabel = "Pareto k (ξ)",
    ylabel = "Frequency",
    title = "PSIS-LOO Pareto k diagnostics"
)
vline!(
    [0.5, 0.7, 1.0],
    linestyle = :dash,
)
savefig("output/pareto_k_$title.png")

[ Info: No source provided for samples; variables are assumed to be from a Markov Chain. If the samples are independent, specify this with keyword argument `source=:other`.


"c:\\Users\\michi\\WorkSpace\\gitwork\\mich2000jp\\hmm\\output\\pareto_k_fp_glm_fin.png"